# Visualisation des résultats

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
## Visualisation des résultats
# Load the test dataset
test_dataset = MyDataset('test.hdf5')
print(batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Chemin vers les modèles sauvergardés
chemin_vers_sauvegarde_model0 = 'model0.pth'
chemin_vers_sauvegarde_model1 = 'model1.pth'
chemin_vers_sauvegarde_model2 = 'model2.pth'
chemin_vers_sauvegarde_model3 = 'model3.pth'
chemin_vers_sauvegarde_model4 = 'model4.pth'
chemin_vers_sauvegarde_modelLSTM = 'modelCNNLSTM.pth'
chemin_vers_sauvegarde_model3_augmented = 'model3_augmented.pth'

# Load the trained models
model0 = torch.load(chemin_vers_sauvegarde_model0, map_location=device, weights_only=False)
model1 = torch.load(chemin_vers_sauvegarde_model1, map_location=device, weights_only=False)
model2 = torch.load(chemin_vers_sauvegarde_model2, map_location=device, weights_only=False)
model3 = torch.load(chemin_vers_sauvegarde_model3, map_location=device, weights_only=False)
model4 = torch.load(chemin_vers_sauvegarde_model4, map_location=device, weights_only=False)
model_lstm = torch.load(chemin_vers_sauvegarde_modelLSTM, map_location=device, weights_only=False)
model3_augmented = torch.load(chemin_vers_sauvegarde_model3_augmented, map_location=device, weights_only=False)

# Set the models to evaluation mode
model0.eval()
model1.eval()
model2.eval()
model3.eval()
model4.eval()
model_lstm.eval()
model3_augmented.eval()

# Move models to the appropriate device
model0.to(device)
model1.to(device)
model2.to(device)
model3.to(device)
model4.to(device)
model_lstm.to(device)
model3_augmented.to(device)

# Initialize accuracy metric
accuracy_metric = Accuracy(task="multiclass", num_classes=num_classes).to(device)

# Initialize lists to store true and predicted labels
true_labels = []
predicted_labels_model0 = []
predicted_labels_model1 = []
predicted_labels_model2 = []
predicted_labels_model3 = []
predicted_labels_model4 = []
predicted_labels_modelLSTM = []
predicted_labels_model3_augmented = []

# Evaluate models
with torch.no_grad():
    for batch_x, batch_y in test_dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        
        # Ensure the input tensor has the correct shape
        if batch_x.shape[1] == 2048:
            batch_x = batch_x.view(batch_x.size(0), 2, -1)
        
        # Collect true labels
        true_labels.extend(batch_y.cpu().numpy())
        
        # Evaluate each model
        for model, predicted_labels in zip(
            [model0, model1, model2, model3, model4, model_lstm, model3_augmented],
            [predicted_labels_model0, predicted_labels_model1, predicted_labels_model2, predicted_labels_model3, predicted_labels_model4, predicted_labels_modelLSTM, predicted_labels_model3_augmented]
        ):
            outputs = model(batch_x)
            _, predicted = torch.max(outputs, 1)
            predicted_labels.extend(predicted.cpu().numpy())


# Convert lists to numpy arrays for easier manipulation and faster computing
true_labels = np.array(true_labels)
predicted_labels_model0 = np.array(predicted_labels_model0)
predicted_labels_model1 = np.array(predicted_labels_model1)
predicted_labels_model2 = np.array(predicted_labels_model2)
predicted_labels_model3 = np.array(predicted_labels_model3)
predicted_labels_model4 = np.array(predicted_labels_model4)
predicted_labels_modelLSTM = np.array(predicted_labels_modelLSTM)
predicted_labels_model3_augmented = np.array(predicted_labels_model3_augmented)

# Calculate accuracy for each model
print(predicted_labels_model0)
accuracy_model0 = accuracy_metric(torch.tensor(predicted_labels_model0, dtype=torch.float32), torch.tensor(true_labels)).item()
accuracy_model1 = accuracy_metric(torch.tensor(predicted_labels_model1), torch.tensor(true_labels)).item()
accuracy_model2 = accuracy_metric(torch.tensor(predicted_labels_model2), torch.tensor(true_labels)).item()
accuracy_model3 = accuracy_metric(torch.tensor(predicted_labels_model3), torch.tensor(true_labels)).item()
accuracy_model4 = accuracy_metric(torch.tensor(predicted_labels_model4), torch.tensor(true_labels)).item()
accuracy_modelLSTM = accuracy_metric(torch.tensor(predicted_labels_modelLSTM), torch.tensor(true_labels)).item()
accuracy_model3_augmented = accuracy_metric(torch.tensor(predicted_labels_model3_augmented), torch.tensor(true_labels)).item()

# Print accuracies
print(f"Model0 Accuracy: {accuracy_model0:.4f}")
print(f"Model1 Accuracy: {accuracy_model1:.4f}")
print(f"Model2 Accuracy: {accuracy_model2:.4f}")
print(f"Model3 Accuracy: {accuracy_model3:.4f}")
print(f"Model4 Accuracy: {accuracy_model4:.4f}")
print(f"ModelLSTM Accuracy: {accuracy_modelLSTM:.4f}")
print(f"Model3 Augmented Accuracy: {accuracy_model3_augmented:.4f}")

# Calculate confusion matrices for each model
#confusion_matrix_model0 = confusion_matrix(true_labels, predicted_labels_model0)
confusion_matrix_model1 = confusion_matrix(true_labels, predicted_labels_model1)
confusion_matrix_model2 = confusion_matrix(true_labels, predicted_labels_model2)
confusion_matrix_model3 = confusion_matrix(true_labels, predicted_labels_model3)
confusion_matrix_model4 = confusion_matrix(true_labels, predicted_labels_model4)
confusion_matrix_modelLSTM = confusion_matrix(true_labels, predicted_labels_modelLSTM)
confusion_matrix_model3_augmented = confusion_matrix(true_labels, predicted_labels_model3_augmented)

# Plot confusion matrices
for model_name, cm in zip(
    ["Model0", "Model1", "Model2", "Model3", "Model4", "ModelLSTM", "Model3 Augmented"],
    #[confusion_matrix_model0, confusion_matrix_model1, confusion_matrix_model2, confusion_matrix_model3, confusion_matrix_model4, confusion_matrix_modelLSTM, confusion_matrix_model3_augmented]
    [confusion_matrix_model1, confusion_matrix_model2, confusion_matrix_model3, confusion_matrix_model4, confusion_matrix_modelLSTM, confusion_matrix_model3_augmented]
):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion Matrix for {model_name}")
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.show()

In [ ]:
# Calculate accuracy vs SNR
accuracies = {
    "model1": [],
    "model2": [],
    "model3": [],
    "model4": [],
    "modelLSTM": [],
    "model3_augmented": []
}
snrs = distinct_snr

for snr in snrs:
    true_labels_snr = []
    predicted_labels_snr = {model: [] for model in accuracies.keys()}
    
    with torch.no_grad():
        for batch_x, batch_y in test_dataloader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            # Ensure the input tensor has the correct shape
            if batch_x.shape[1] == 2048:
                batch_x = batch_x.view(batch_x.size(0), 2, -1)
            
            # Collect true labels
            true_labels_snr.extend(batch_y.cpu().numpy())
            
            # Evaluate each model
            for model_name, model in zip(accuracies.keys(), 
                                         [model1, model2, model3, model4, model_lstm, model3_augmented]):
                outputs = model(batch_x)
                _, predicted = torch.max(outputs, 1)
                predicted_labels_snr[model_name].extend(predicted.cpu().numpy())

        # Calculate accuracy for each model at the current SNR
        for model_name in accuracies.keys():
            print(model_name)
            acc = accuracy_metric(torch.tensor(predicted_labels_snr[model_name], dtype=torch.float32), 
                                  torch.tensor(true_labels_snr))
            accuracies[model_name].append(acc.item())

# Plot accuracy vs SNR
plt.figure(figsize=(10, 6))
for model_name, model_accuracies in accuracies.items():
    plt.plot(snrs, model_accuracies, label=model_name)
plt.xlabel("SNR (dB)")
plt.ylabel("Accuracy")
plt.title("Accuracy vs SNR for Different Models")
plt.legend()
plt.grid()
plt.show()

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

complexities = {
    "Model0": count_parameters(model0),
    "Model1": count_parameters(model1),
    "Model2": count_parameters(model2),
    "Model3": count_parameters(model3),
    "Model4": count_parameters(model4),
    "ModelLSTM": count_parameters(model_lstm),
    "Model3 Augmented": count_parameters(model3_augmented)
}

print("Model Complexities:")
for model_name, num_params in complexities.items():
    print(f"{model_name}: {num_params} parameters")